<img style="float: right" src="img/saturn.png" width="300" />

# Machine Learning on Big Data with Dask

## Train models!

Now that we can process big data as illustrated in [04-large-dataset.ipynb](04-large-dataset.ipynb), let's train some models!

## Initialize cluster and load dataset

The [05-setup.py](05-setup.py) contains the same data loading code from [04-large-dataset.ipynb](04-large-dataset.ipynb). After running that, we have the following objects available:

- `client`: Dask client
- `taxi_feat`: Taxi dataframe with engineering features

In [ ]:
%run setup.py

In [ ]:
client

In [ ]:
taxi_feat

### Exercise

Persist the `taxi_feat` dataframe to make sure subsequent operations are fast. Block until it's completely finished persisting.

In [ ]:
%%time
<<< FILL IN >>>

In [ ]:
%%time
taxi_feat = taxi_feat.persist()
_ = wait(taxi_feat)

## Train model with large dataset

First, we need to split our `taxi_feat` DataFrame into train/test sets.

### Exercise

Use the [`dask_ml.model_selection.train_test_split` function](https://ml.dask.org/modules/generated/dask_ml.model_selection.train_test_split.html) to split into train and test sets. Use 30% of the data for the test set.

> Hint: the `dask_ml` function works the same as the `sklearn` function

In [ ]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = <<< FILL IN >>>

In [ ]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    taxi_feat[features], 
    taxi_feat[label], 
    test_size=0.3,
    random_state=42
)

In [ ]:
len(X_train), len(y_train)

In [ ]:
len(X_test), len(y_test)

In [ ]:
X_train.describe().compute()

--- 

## Dask ML models

The dask-ml package has parallel implementations of machine learning algorithms that do not have parallel implementations in scikit-learn or other packages. These currently cover linear models and clustering.

In [ ]:
from sklearn.pipeline import Pipeline

from dask_ml.linear_model import LinearRegression
from dask_ml.preprocessing import StandardScaler
from dask_ml.metrics import mean_squared_error

lr = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('clf', LinearRegression(penalty='l2', max_iter=100)),
])

Dask-ml's `LinearRegression` model requires arrays as inputs rather than dataframes, so we convert them to arrays. Similar to the relationship between a pandas dataframe and a Dask dataframe, Dask arrays are collections of numpy arrays that conform to a similar API.

With pandas dataframes, we could call `.values` to get a numpy array. Dask needs more information about chunk sizes of the underlying numpy arrays, so we need to do `.to_dask_array(lengths=True)`.

In [ ]:
X_train_arr = X_train.to_dask_array(lengths=True)
X_test_arr = X_test.to_dask_array(lengths=True)
y_train_arr = y_train.to_dask_array(lengths=True)
y_test_arr = y_test.to_dask_array(lengths=True)

In [ ]:
X_train_arr

Due to Dask's lazy evaluation, these dataframes have not been computed yet. To ensure the rest of our ML code runs quickly, lets kick off computation on the cluster by calling `persist()` on the arrays. Note that there is a `dask.persist` function that accepts multiple objects rather than calling `.persist()` individually. This is helpful for objects that share upstream tasks - Dask will avoid re-computing the shared tasks.

In [ ]:
%%time
X_train_arr, X_test_arr, y_train_arr, y_test_arr = dask.persist(
    X_train_arr, X_test_arr, y_train_arr, y_test_arr,
)
_ = wait(X_train_arr)

### Exercise

Train the `lr` model with `X_train_arr` and `y_train_arr` as input.

> Note: this will take a few minutes because we are training with a pretty large dataset. You can scale up your cluster if you want it to execute faster!

In [ ]:
%%time
lr_fitted = <<< FILL IN >>>

In [ ]:
%%time

lr_fitted = lr.fit(
    X_train_arr,
    y_train_arr,
)

--- 

In [ ]:
lr_preds = lr_fitted.predict(X_test_arr)
mean_squared_error(y_test_arr, lr_preds, squared=False)

## XGBoost

XGBoost supports distributed model training across Dask clusters in the `xgboost.dask` module. For a deeper dive into how this works, check out our tutorial on [XGBoost Training with Dask](https://www.saturncloud.io/docs/tutorials/xgboost/).

In [ ]:
from xgboost.dask import DaskXGBRegressor

xgb = DaskXGBRegressor(
    objective="reg:squarederror",
    tree_method='hist',
    learning_rate=0.1,
    max_depth=5,
    n_estimators=100,
)

In [ ]:
%%time

xgb_fitted = xgb.fit(
    X_train_arr,
    y_train_arr,
)

In [ ]:
xgb_preds = xgb_fitted.predict(X_test_arr)
mean_squared_error(y_test_arr, xgb_preds, squared=False)

XGBoost also supports GPU acceleration across one or more GPUs in a Dask cluster. Stay tuned for a future workshop on all things Dask+XGBoost!